# Прием и обработка твитов микробатчем

## Инициализация

In [56]:
import org.apache.spark.sql.types.{StructType, StringType, IntegerType, TimestampType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.toree.kernel.api
import java.util.Calendar

In [57]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.4


## Чтение модели

In [58]:
val modelPath = "/home/jovyan/models/spark-ml-model"
val model = PipelineModel.load(modelPath)

modelPath = /home/jovyan/models/spark-ml-model
model = pipeline_6cb85880407d


pipeline_6cb85880407d

## Определяем схему и инициируем потоковый датафрейм

In [68]:
val inputStreamPath = "/home/jovyan/work/events-stream"
val outputStreamPath = "/home/jovyan/work/events-stream-out"
val modelPath = "/home/jovyan/models/spark-ml-model"

// Определяем udf для получения probability по 0 и 1
val getProbability =
    udf(
        (prediction: org.apache.spark.ml.linalg.Vector, pos: Integer) =>
        {
            prediction(pos)
        }
    )

val dataSchema = new StructType()
    .add("tweet", StringType)
    .add("hiddentargetclue", IntegerType)
    .add("timestamp", TimestampType)

/*
                    .select(
                        $"arrived_key",
                        $"tweet",
                        // $"hiddentargetclue",
                        (getProbability($"probability",lit(0))).alias("Negative Probability")
                    )
*/

/*
                model.transform(batchDF)
                    .select(
                        $"timestamp",
                        $"tweets"
                    )
*/

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)
    .withWatermark("timestamp", "10 seconds")
    .groupBy(
        window($"timestamp", "10 seconds", "5 seconds").alias("timestamp")
    )
    .agg(count("*")).alias("tweets")

//val wStream = inputDF.writeStream
//    .option("checkpointLocation", outputStreamPath)
//.start(outputStreamPath)

inputStreamPath = /home/jovyan/work/events-stream
outputStreamPath = /home/jovyan/work/events-stream-out
modelPath = /home/jovyan/models/spark-ml-model
getProbability = UserDefinedFunction(<function2>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7, IntegerType)))
dataSchema = StructType(StructField(tweet,StringType,true), StructField(hiddentargetclue,IntegerType,true), StructField(timestamp,TimestampType,true))
inputDF = [timestamp: struct<start: timestamp, end: timestamp>, count(1): bigint]


[timestamp: struct<start: timestamp, end: timestamp>, count(1): bigint]

In [ ]:
//inputDF.show()

## Глобальная переменная для удобства просмотра датасета в отдельной ячейке

In [60]:
var globDF:DataFrame = null

globDF: org.apache.spark.sql.DataFrame = null


## Внимание!
- К сожалению, я не нашел внятной доки по апи ядра toree, чтобы нормально выводить оперативно изменяющийся датасет в этом ноутбуке
- В связи с этим см. след. пункт
- В блоке приема твитов ниже вывод датасета для удобства просмотра осуществляется через глобальную переменную globDF, которая просматривается при помощи выполнения блока, следующего за приемом твитов

## Микробатч приема твитов

In [69]:
var fRuns = 0

// Микробатч для вывода результата предсказания
// Выводится вероятность негативного твита
// В задании написано, что это последняя колонка, но она здесь вроде первая (в позиции 0)
val stream = inputDF.writeStream.foreachBatch {
    (batchDF: DataFrame, batchId: Long) => {
        try {
            fRuns += 1
            print(s"${Calendar.getInstance().toInstant} - loaded from the events stream $fRuns times"+13.toChar)
            // Применяем модель и получаем соотв. датасет с предсказаниями
//            globDF = 
//                model.transform(batchDF)
//                    .select(
//                        $"timestamp",
//                        $"tweets"
//                    )
            globDF = batchDF
        } catch {
            case e:Throwable => {
                print(e.getMessage.replaceAll("\n"," "))
                print(13.toChar)
            }
        }
    }
}.start()

fRuns = 0
stream = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@64584c0a


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@64584c0a

## Блок просмотра результата
- Этот блок предназначен для просмотра результата потоковой обработки твитов и применения модели из предыдущего блока
- Каждый раз при выполнении этого блока будет выводиться оперативное состояние датасета из блока выше

In [ ]:
%%dataframe --limit=100
globDF

## Останов чтения потока

In [72]:
globDF = null
stream.stop()

globDF: org.apache.spark.sql.DataFrame = null
